<!--NAVIGATION-->
< [Handling Missing Data](03.04-Missing-Values.ipynb) | [Contents](Index.ipynb) | [Combining Datasets: Concat and Append](03.06-Concat-And-Append.ipynb) >

# Hierarchical Indexing

Up to this point we've been focused primarily on one-dimensional and two-dimensional data, stored in Pandas ``Series`` and ``DataFrame`` objects, respectively.
Often it is useful to go beyond this and store higher-dimensional data–that is, data indexed by more than one or two keys.
While Pandas does provide ``Panel`` and ``Panel4D`` objects that natively handle three-dimensional and four-dimensional data (see [Aside: Panel Data](#Aside:-Panel-Data)), a far more common pattern in practice is to make use of *hierarchical indexing* (also known as *multi-indexing*) to incorporate multiple index *levels* within a single index.
In this way, higher-dimensional data can be compactly represented within the familiar one-dimensional ``Series`` and two-dimensional ``DataFrame`` objects.

In this section, we'll explore the direct creation of ``MultiIndex`` objects, considerations when indexing, slicing, and computing statistics across multiply indexed data, and useful routines for converting between simple and hierarchically indexed representations of your data.

We begin with the standard imports:

In [1]:
import pandas as pd
import numpy as np

## A Multiply Indexed Series

Let's start by considering how we might represent two-dimensional data within a one-dimensional ``Series``.
For concreteness, we will consider a series of data where each point has a character and numerical key.

### The bad way

Suppose you would like to track data about states from two different years.
Using the Pandas tools we've already covered, you might be tempted to simply use Python tuples as keys:

In [2]:
Se = pd.Series(np.random.rand(5),index=['a','b','c','d','e'])
Se

a    0.493642
b    0.997207
c    0.576604
d    0.360905
e    0.829092
dtype: float64

In [3]:
Se[0]

0.49364190774880645

In [4]:
Se['a']

0.49364190774880645

In [5]:
# Index In Series
index = [('Andhara Pradesh', 2000), ('Andhara Pradesh', 2010),
         ('Tamil Nadu', 2000), ('Tamil Nadu', 2010),
         ('Karnataka', 2000), ('Karnataka', 2010)]

# Data In `Series`
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]

In [6]:
Se = pd.Series(populations, index=index)
Se

(Andhara Pradesh, 2000)    33871648
(Andhara Pradesh, 2010)    37253956
(Tamil Nadu, 2000)         18976457
(Tamil Nadu, 2010)         19378102
(Karnataka, 2000)          20851820
(Karnataka, 2010)          25145561
dtype: int64

With this indexing scheme, you can straightforwardly index or slice the series based on this multiple index:

In [7]:
Se[('Andhara Pradesh', 2010)]

37253956

In [8]:
Se[('Andhara Pradesh', 2000)]

33871648

In [9]:
Se[0]

33871648

In [10]:
Se[1]

37253956

In [11]:
Se[1:4]

(Andhara Pradesh, 2010)    37253956
(Tamil Nadu, 2000)         18976457
(Tamil Nadu, 2010)         19378102
dtype: int64

In [12]:
Se[('Andhara Pradesh', 2010):('Tamil Nadu', 2000)]

(Andhara Pradesh, 2010)    37253956
(Tamil Nadu, 2000)         18976457
dtype: int64

But the convenience ends there. For example, if you need to select all values from 2010, you'll need to do some messy (and potentially slow) munging to make it happen:

In [13]:
Se[[i for i in Se.index if i[1] == 2010]]

(Andhara Pradesh, 2010)    37253956
(Tamil Nadu, 2010)         19378102
(Karnataka, 2010)          25145561
dtype: int64

In [14]:
for i in Se.index:
    if i[1]==2010:
        print(i,Se[i])

('Andhara Pradesh', 2010) 37253956
('Tamil Nadu', 2010) 19378102
('Karnataka', 2010) 25145561


This produces the desired result, but is not as clean (or as efficient for large datasets) as the slicing syntax we've grown to love in Pandas.

### The Better Way: Pandas MultiIndex
Fortunately, Pandas provides a better way.
Our tuple-based indexing is essentially a rudimentary multi-index, and the Pandas ``MultiIndex`` type gives us the type of operations we wish to have.
We can create a multi-index from the tuples as follows:

In [15]:
print(index)

[('Andhara Pradesh', 2000), ('Andhara Pradesh', 2010), ('Tamil Nadu', 2000), ('Tamil Nadu', 2010), ('Karnataka', 2000), ('Karnataka', 2010)]


In [16]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('Andhara Pradesh', 2000),
            ('Andhara Pradesh', 2010),
            (     'Tamil Nadu', 2000),
            (     'Tamil Nadu', 2010),
            (      'Karnataka', 2000),
            (      'Karnataka', 2010)],
           )

In [18]:
index[2]

('Tamil Nadu', 2000)

In [19]:
dict(index)

{'Andhara Pradesh': 2010, 'Tamil Nadu': 2010, 'Karnataka': 2010}

Notice that the ``MultiIndex`` contains multiple *levels* of indexing–in this case, the state names and the years, as well as multiple *labels* for each data point which encode these levels.

If we re-index our series with this ``MultiIndex``, we see the hierarchical representation of the data:

In [20]:
Se

(Andhara Pradesh, 2000)    33871648
(Andhara Pradesh, 2010)    37253956
(Tamil Nadu, 2000)         18976457
(Tamil Nadu, 2010)         19378102
(Karnataka, 2000)          20851820
(Karnataka, 2010)          25145561
dtype: int64

In [21]:
pop = Se.reindex(index)
pop

Andhara Pradesh  2000    33871648
                 2010    37253956
Tamil Nadu       2000    18976457
                 2010    19378102
Karnataka        2000    20851820
                 2010    25145561
dtype: int64

In [22]:
pop[1]

37253956

In [23]:
pop[0]

33871648

In [24]:
pop['Andhara Pradesh'] 

2000    33871648
2010    37253956
dtype: int64

Here the first two columns of the ``Series`` representation show the multiple index values, while the third column shows the data.
Notice that some entries are missing in the first column: in this multi-index representation, any blank entry indicates the same value as the line above it.

Now to access all data for which the second index is 2010, we can simply use the Pandas slicing notation:

In [25]:
pop[:,2010]

Andhara Pradesh    37253956
Tamil Nadu         19378102
Karnataka          25145561
dtype: int64

In [26]:
pop[:,2000]

Andhara Pradesh    33871648
Tamil Nadu         18976457
Karnataka          20851820
dtype: int64

In [27]:
pop.shape

(6,)

The result is a singly indexed array with just the keys we're interested in.
This syntax is much more convenient (and the operation is much more efficient!) than the home-spun tuple-based multi-indexing solution that we started with.
We'll now further discuss this sort of indexing operation on hieararchically indexed data.

### MultiIndex as extra dimension

You might notice something else here: we could easily have stored the same data using a simple ``DataFrame`` with index and column labels.
In fact, Pandas is built with this equivalence in mind. The ``unstack()`` method will quickly convert a multiply indexed ``Series`` into a conventionally indexed ``DataFrame``:

In [28]:
pop_df = pop.unstack()
pop_df

,2000,2010
Andhara Pradesh,33871648,37253956
Karnataka,20851820,25145561
Tamil Nadu,18976457,19378102


Naturally, the ``stack()`` method provides the opposite operation:

In [29]:
pop_df.stack()

Andhara Pradesh  2000    33871648
                 2010    37253956
Karnataka        2000    20851820
                 2010    25145561
Tamil Nadu       2000    18976457
                 2010    19378102
dtype: int64

Seeing this, you might wonder why would we would bother with hierarchical indexing at all.
The reason is simple: just as we were able to use multi-indexing to represent two-dimensional data within a one-dimensional ``Series``, we can also use it to represent data of three or more dimensions in a ``Series`` or ``DataFrame``.
Each extra level in a multi-index represents an extra dimension of data; taking advantage of this property gives us much more flexibility in the types of data we can represent. Concretely, we might want to add another column of demographic data for each state at each year (say, population under 18) ; with a ``MultiIndex`` this is as easy as adding another column to the ``DataFrame``:

In [30]:
pop

Andhara Pradesh  2000    33871648
                 2010    37253956
Tamil Nadu       2000    18976457
                 2010    19378102
Karnataka        2000    20851820
                 2010    25145561
dtype: int64

In [31]:
pop_df = pd.DataFrame({'total': pop,
                       'under_18_Age': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under_18_Age
Andhara Pradesh 2000  33871648       9267089
                2010  37253956       9284094
Tamil Nadu      2000  18976457       4687374
                2010  19378102       4318033
Karnataka       2000  20851820       5906301
                2010  25145561       6879014

In addition, all the ufuncs and other functionality discussed in [Operating on Data in Pandas](03.03-Operations-in-Pandas.ipynb) work with hierarchical indices as well.
Here we compute the fraction of people under 18 by year, given the above data:

In [32]:
f_u18 = pop_df['under_18_Age'] / pop_df['total']
f_u18.unstack()

,2000,2010
Andhara Pradesh,0.273594,0.249211
Karnataka,0.283251,0.273568
Tamil Nadu,0.247010,0.222831


This allows us to easily and quickly manipulate and explore even high-dimensional data.

## Methods of MultiIndex Creation

The most straightforward way to construct a multiply indexed ``Series`` or ``DataFrame`` is to simply pass a list of two or more index arrays to the constructor. For example:

In [33]:
df = pd.DataFrame(np.random.rand(16, 4),
                  index=[['a', 'a','a','a', 'b','b', 'b','b', 'c', 'c', 'c','c','d','d','d','d'], 
                         [1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4]],
                  columns=['Data1', 'Data2','Data3','Data4'])
df

Data1     Data2     Data3     Data4
a 1  0.127245  0.426823  0.977224  0.886285
  2  0.883758  0.454783  0.303977  0.571477
  3  0.246610  0.721823  0.753888  0.398409
  4  0.402302  0.730547  0.120078  0.679271
b 1  0.483192  0.121341  0.476543  0.584979
  2  0.375625  0.415801  0.654932  0.364417
  3  0.660278  0.581944  0.396214  0.213527
  4  0.995530  0.499918  0.641386  0.817324
c 1  0.792633  0.649781  0.775035  0.071087
  2  0.630369  0.137935  0.175531  0.281156
  3  0.582394  0.069108  0.945945  0.916976
  4  0.579755  0.311497  0.624431  0.411520
d 1  0.031137  0.431782  0.343570  0.070588
  2  0.820812  0.431114  0.289942  0.025803
  3  0.434420  0.045460  0.001392  0.409371
  4  0.159235  0.151615  0.503034  0.071488

The work of creating the ``MultiIndex`` is done in the background.

Similarly, if you pass a dictionary with appropriate tuples as keys, Pandas will automatically recognize this and use a ``MultiIndex`` by default:

In [34]:
data = {('Andhar Pradesh', 2000): 33871648,
        ('Andhar Pradesh', 2010): 37253956,
        ('Tamil Nadu', 2000): 20851820,
        ('Tamil Nadu', 2010): 25145561,
        ('Karnataka', 2000): 18976457,
        ('Karnataka', 2010): 19378102}
pd.Series(data)

Andhar Pradesh  2000    33871648
                2010    37253956
Tamil Nadu      2000    20851820
                2010    25145561
Karnataka       2000    18976457
                2010    19378102
dtype: int64

Nevertheless, it is sometimes useful to explicitly create a ``MultiIndex``; we'll see a couple of these methods here.

### Explicit MultiIndex constructors

For more flexibility in how the index is constructed, you can instead use the class method constructors available in the ``pd.MultiIndex``.
For example, as we did before, you can construct the ``MultiIndex`` from a simple list of arrays giving the index values within each level:

In [35]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

You can construct it from a list of tuples giving the multiple index values of each point:

In [36]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

You can even construct it from a Cartesian product of single indices:

In [37]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

Similarly, you can construct the ``MultiIndex`` directly using its internal encoding by passing ``levels`` (a list of lists containing available index values for each level) and ``labels`` (a list of lists that reference these labels):

In [38]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [39]:
arrays = [[1, 1, 2, 2], ['red', 'blue', 'red', 'blue']]
pd.MultiIndex.from_arrays(arrays, names=('number', 'color'))

MultiIndex([(1,  'red'),
            (1, 'blue'),
            (2,  'red'),
            (2, 'blue')],
           names=['number', 'color'])

#### Any of these objects can be passed as the ``index`` argument when creating a ``Series`` or ``Dataframe``, or be passed to the ``reindex`` method of an existing ``Series`` or ``DataFrame``.

### MultiIndex level names

Sometimes it is convenient to name the levels of the ``MultiIndex``.
This can be accomplished by passing the ``names`` argument to any of the above ``MultiIndex`` constructors, or by setting the ``names`` attribute of the index after the fact:

In [41]:
pop

Andhara Pradesh  2000    33871648
                 2010    37253956
Tamil Nadu       2000    18976457
                 2010    19378102
Karnataka        2000    20851820
                 2010    25145561
dtype: int64

In [148]:
pop.index.names = ['state', 'year']
pop

state            year
Andhara Pradesh  2000    33871648
                 2010    37253956
Tamil Nadu       2000    18976457
                 2010    19378102
Karnataka        2000    20851820
                 2010    25145561
dtype: int64

With more involved datasets, this can be a useful way to keep track of the meaning of various index values.

### MultiIndex for columns

In a ``DataFrame``, the rows and columns are completely symmetric, and just as the rows can have multiple levels of indices, the columns can have multiple levels as well.
Consider the following, which is a mock-up of some (somewhat realistic) medical data:

In [42]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Raju', 'Rani', 'Divya'], ['BP', 'Temp']],
                                     names=['subject', 'type'])
# mock some data
data = np.round(np.random.rand(4, 6), 1)
data

array([[0.2, 0.3, 0.6, 0.4, 0.9, 1. ],
       [0.7, 0.3, 0.6, 0.4, 0.5, 0.1],
       [0.8, 0.7, 1. , 0.2, 0.3, 0.3],
       [0.5, 0.7, 1. , 0.9, 0.9, 0.4]])

In [150]:
data[:, ::2] *= 10 # data[:, ::2]*2
data

array([[ 8. ,  0.1,  2. ,  0.1,  6. ,  0.9],
       [ 1. ,  1. ,  7. ,  0.3,  2. ,  1. ],
       [ 4. ,  0.2,  8. ,  1. , 10. ,  0.4],
       [ 8. ,  0. ,  6. ,  0.8, 10. ,  0.7]])

In [151]:
data += 37
data

array([[45. , 37.1, 39. , 37.1, 43. , 37.9],
       [38. , 38. , 44. , 37.3, 39. , 38. ],
       [41. , 37.2, 45. , 38. , 47. , 37.4],
       [45. , 37. , 43. , 37.8, 47. , 37.7]])

In [152]:
# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject     Raju        Rani       Divya      
type          BP  Temp    BP  Temp    BP  Temp
year visit                                    
2013 1      45.0  37.1  39.0  37.1  43.0  37.9
     2      38.0  38.0  44.0  37.3  39.0  38.0
2014 1      41.0  37.2  45.0  38.0  47.0  37.4
     2      45.0  37.0  43.0  37.8  47.0  37.7

Here we see where the multi-indexing for both rows and columns can come in *very* handy.
This is fundamentally four-dimensional data, where the dimensions are the subject, the measurement type, the year, and the visit number.
With this in place we can, for example, index the top-level column by the person's name and get a full ``DataFrame`` containing just that person's information:

In [153]:
health_data['Rani']

type          BP  Temp
year visit            
2013 1      39.0  37.1
     2      44.0  37.3
2014 1      45.0  38.0
     2      43.0  37.8

In [154]:
health_data['Rani']['BP']

year  visit
2013  1        39.0
      2        44.0
2014  1        45.0
      2        43.0
Name: BP, dtype: float64

For complicated records containing multiple labeled measurements across multiple times for many subjects (people, countries, cities, etc.) use of hierarchical rows and columns can be extremely convenient!

## Indexing and Slicing a MultiIndex

Indexing and slicing on a ``MultiIndex`` is designed to be intuitive, and it helps if you think about the indices as added dimensions.
We'll first look at indexing multiply indexed ``Series``, and then multiply-indexed ``DataFrame``s.

### Multiply indexed Series

Consider the multiply indexed ``Series`` of state populations we saw earlier:

In [155]:
pop

state            year
Andhara Pradesh  2000    33871648
                 2010    37253956
Tamil Nadu       2000    18976457
                 2010    19378102
Karnataka        2000    20851820
                 2010    25145561
dtype: int64

We can access single elements by indexing with multiple terms:

In [156]:
pop['Andhara Pradesh', 2000]

33871648

In [157]:
pop.iloc[0]

33871648

In [158]:
pop.iloc[1]

37253956

In [159]:
pop.loc['Andhara Pradesh']

year
2000    33871648
2010    37253956
dtype: int64

The ``MultiIndex`` also supports *partial indexing*, or indexing just one of the levels in the index.
The result is another ``Series``, with the lower-level indices maintained:

In [160]:
pop['Andhara Pradesh']

year
2000    33871648
2010    37253956
dtype: int64

In [161]:
pop[['Andhara Pradesh','Karnataka']]

state            year
Andhara Pradesh  2000    33871648
                 2010    37253956
Karnataka        2000    20851820
                 2010    25145561
dtype: int64

Partial slicing is available as well, as long as the ``MultiIndex`` is sorted (see discussion in [Sorted and Unsorted Indices](Sorted-and-Unsorted-Indices)):

In [162]:
pop.loc['Andhara Pradesh']

year
2000    33871648
2010    37253956
dtype: int64

With sorted indices, partial indexing can be performed on lower levels by passing an empty slice in the first index:

In [163]:
pop[:, 2010]

state
Andhara Pradesh    37253956
Tamil Nadu         19378102
Karnataka          25145561
dtype: int64

In [164]:
pop

state            year
Andhara Pradesh  2000    33871648
                 2010    37253956
Tamil Nadu       2000    18976457
                 2010    19378102
Karnataka        2000    20851820
                 2010    25145561
dtype: int64

Other types of indexing and selection (discussed in [Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb)) work as well; for example, selection based on Boolean masks:

In [165]:
pop > 22000000

state            year
Andhara Pradesh  2000     True
                 2010     True
Tamil Nadu       2000    False
                 2010    False
Karnataka        2000    False
                 2010     True
dtype: bool

In [166]:
pop[pop > 22000000]

state            year
Andhara Pradesh  2000    33871648
                 2010    37253956
Karnataka        2010    25145561
dtype: int64

In [167]:
pop[pop < 22000000]

state       year
Tamil Nadu  2000    18976457
            2010    19378102
Karnataka   2000    20851820
dtype: int64

Selection based on fancy indexing also works:

In [168]:
pop[['Andhara Pradesh','Karnataka']]

state            year
Andhara Pradesh  2000    33871648
                 2010    37253956
Karnataka        2000    20851820
                 2010    25145561
dtype: int64

### Multiply indexed DataFrames

A multiply indexed ``DataFrame`` behaves in a similar manner.
Consider our toy medical ``DataFrame`` from before:

In [169]:
health_data

subject     Raju        Rani       Divya      
type          BP  Temp    BP  Temp    BP  Temp
year visit                                    
2013 1      45.0  37.1  39.0  37.1  43.0  37.9
     2      38.0  38.0  44.0  37.3  39.0  38.0
2014 1      41.0  37.2  45.0  38.0  47.0  37.4
     2      45.0  37.0  43.0  37.8  47.0  37.7

Remember that columns are primary in a ``DataFrame``, and the syntax used for multiply indexed ``Series`` applies to the columns.
For example, we can recover Guido's heart rate data with a simple operation:

In [170]:
health_data['Raju', 'BP']

year  visit
2013  1        45.0
      2        38.0
2014  1        41.0
      2        45.0
Name: (Raju, BP), dtype: float64

In [171]:
health_data['Raju', 'BP']>40.0

year  visit
2013  1         True
      2        False
2014  1         True
      2         True
Name: (Raju, BP), dtype: bool

Also, as with the single-index case, we can use the ``loc``, ``iloc``, and ``ix`` indexers introduced in [Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb). For example:

In [172]:
                # Row # Col
health_data.iloc[:2,:-2]

subject     Raju        Rani      
type          BP  Temp    BP  Temp
year visit                        
2013 1      45.0  37.1  39.0  37.1
     2      38.0  38.0  44.0  37.3

In [173]:
health_data.iloc[2:,:-2]

subject     Raju        Rani      
type          BP  Temp    BP  Temp
year visit                        
2014 1      41.0  37.2  45.0  38.0
     2      45.0  37.0  43.0  37.8

These indexers provide an array-like view of the underlying two-dimensional data, but each individual index in ``loc`` or ``iloc`` can be passed a tuple of multiple indices. For example:

In [174]:
health_data.loc[:, ('Rani', 'BP')]

year  visit
2013  1        39.0
      2        44.0
2014  1        45.0
      2        43.0
Name: (Rani, BP), dtype: float64

Working with slices within these index tuples is not especially convenient; trying to create a slice within a tuple will lead to a syntax error:

In [175]:
health_data.loc[(2, 1), (:,'BP')]

SyntaxError: invalid syntax (<ipython-input-175-4c2795ad5415>, line 1)

You could get around this by building the desired slice explicitly using Python's built-in ``slice()`` function, but a better way in this context is to use an ``IndexSlice`` object, which Pandas provides for precisely this situation.
For example:

In [177]:
idx = pd.IndexSlice
health_data.loc[idx[2013, :1], idx[:, 'BP']]

,subject,Raju,Rani,Divya
,type,BP,BP,BP
year,visit,,,
2013,1,45.0,39.0,43.0


There are so many ways to interact with data in multiply indexed ``Series`` and ``DataFrame``s, and as with many tools in this book the best way to become familiar with them is to try them out!

## Rearranging Multi-Indices

One of the keys to working with multiply indexed data is knowing how to effectively transform the data.
There are a number of operations that will preserve all the information in the dataset, but rearrange it for the purposes of various computations.
We saw a brief example of this in the ``stack()`` and ``unstack()`` methods, but there are many more ways to finely control the rearrangement of data between hierarchical indices and columns, and we'll explore them here.

### Sorted and unsorted indices

Earlier, we briefly mentioned a caveat, but we should emphasize it more here.
*Many of the ``MultiIndex`` slicing operations will fail if the index is not sorted.*
Let's take a look at this here.

We'll start by creating some simple multiply indexed data where the indices are *not lexographically sorted*:

In [178]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.595969
      2      0.983082
c     1      0.346015
      2      0.826726
b     1      0.810267
      2      0.692910
dtype: float64

If we try to take a partial slice of this index, it will result in an error:

In [179]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


Although it is not entirely clear from the error message, this is the result of the MultiIndex not being sorted.
For various reasons, partial slices and other similar operations require the levels in the ``MultiIndex`` to be in sorted (i.e., lexographical) order.
Pandas provides a number of convenience routines to perform this type of sorting; examples are the ``sort_index()`` and ``sortlevel()`` methods of the ``DataFrame``.
We'll use the simplest, ``sort_index()``, here:

In [180]:
data = data.sort_index()
data

char  int
a     1      0.595969
      2      0.983082
b     1      0.810267
      2      0.692910
c     1      0.346015
      2      0.826726
dtype: float64

In [181]:
data['a':'b']

char  int
a     1      0.595969
      2      0.983082
b     1      0.810267
      2      0.692910
dtype: float64

In [182]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

With the index sorted in this way, partial slicing will work as expected:

### Stacking and unstacking indices

As we saw briefly before, it is possible to convert a dataset from a stacked multi-index to a simple two-dimensional representation, optionally specifying the level to use:

In [183]:
pop

state            year
Andhara Pradesh  2000    33871648
                 2010    37253956
Tamil Nadu       2000    18976457
                 2010    19378102
Karnataka        2000    20851820
                 2010    25145561
dtype: int64

In [184]:
pop.unstack()

year,2000,2010
state,,
Andhara Pradesh,33871648,37253956
Karnataka,20851820,25145561
Tamil Nadu,18976457,19378102


In [185]:
pop.unstack(level=0)

state,Andhara Pradesh,Karnataka,Tamil Nadu
year,,,
2000,33871648,20851820,18976457
2010,37253956,25145561,19378102


In [186]:
pop.unstack(level=1)

year,2000,2010
state,,
Andhara Pradesh,33871648,37253956
Karnataka,20851820,25145561
Tamil Nadu,18976457,19378102


The opposite of ``unstack()`` is ``stack()``, which here can be used to recover the original series:

In [187]:
pop.unstack().stack()

state            year
Andhara Pradesh  2000    33871648
                 2010    37253956
Karnataka        2000    20851820
                 2010    25145561
Tamil Nadu       2000    18976457
                 2010    19378102
dtype: int64

### Index setting and resetting

Another way to rearrange hierarchical data is to turn the index labels into columns; this can be accomplished with the ``reset_index`` method.
Calling this on the population dictionary will result in a ``DataFrame`` with a *state* and *year* column holding the information that was formerly in the index.
For clarity, we can optionally specify the name of the data for the column representation:

In [188]:
pop

state            year
Andhara Pradesh  2000    33871648
                 2010    37253956
Tamil Nadu       2000    18976457
                 2010    19378102
Karnataka        2000    20851820
                 2010    25145561
dtype: int64

In [189]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,Andhara Pradesh,2000,33871648
1,Andhara Pradesh,2010,37253956
2,Tamil Nadu,2000,18976457
3,Tamil Nadu,2010,19378102
4,Karnataka,2000,20851820
5,Karnataka,2010,25145561


Often when working with data in the real world, the raw input data looks like this and it's useful to build a ``MultiIndex`` from the column values.
This can be done with the ``set_index`` method of the ``DataFrame``, which returns a multiply indexed ``DataFrame``:

In [190]:
pop_flat.set_index(['state', 'year'])

population
state           year            
Andhara Pradesh 2000    33871648
                2010    37253956
Tamil Nadu      2000    18976457
                2010    19378102
Karnataka       2000    20851820
                2010    25145561

In practice, I find this type of reindexing to be one of the more useful patterns when encountering real-world datasets.

## Data Aggregations on Multi-Indices

We've previously seen that Pandas has built-in data aggregation methods, such as ``mean()``, ``sum()``, and ``max()``.
For hierarchically indexed data, these can be passed a ``level`` parameter that controls which subset of the data the aggregate is computed on.

For example, let's return to our health data:

In [191]:
health_data

subject     Raju        Rani       Divya      
type          BP  Temp    BP  Temp    BP  Temp
year visit                                    
2013 1      45.0  37.1  39.0  37.1  43.0  37.9
     2      38.0  38.0  44.0  37.3  39.0  38.0
2014 1      41.0  37.2  45.0  38.0  47.0  37.4
     2      45.0  37.0  43.0  37.8  47.0  37.7

Perhaps we'd like to average-out the measurements in the two visits each year. We can do this by naming the index level we'd like to explore, in this case the year:

In [192]:
data_mean = health_data.mean(level='year')
data_mean

subject  Raju         Rani       Divya       
type       BP   Temp    BP  Temp    BP   Temp
year                                         
2013     41.5  37.55  41.5  37.2  41.0  37.95
2014     43.0  37.10  44.0  37.9  47.0  37.55

By further making use of the ``axis`` keyword, we can take the mean among levels on the columns as well:

In [193]:
data_mean.mean(axis=1, level='type')

type,BP,Temp
year,,
2013,41.333333,37.566667
2014,44.666667,37.516667


Thus in two lines, we've been able to find the average heart rate and temperature measured among all subjects in all visits each year.
This syntax is actually a short cut to the ``GroupBy`` functionality, which we will discuss in [Aggregation and Grouping](03.08-Aggregation-and-Grouping.ipynb).
While this is a toy example, many real-world datasets have similar hierarchical structure.

## Aside: Panel Data

Pandas has a few other fundamental data structures that we have not yet discussed, namely the ``pd.Panel`` and ``pd.Panel4D`` objects.
These can be thought of, respectively, as three-dimensional and four-dimensional generalizations of the (one-dimensional) ``Series`` and (two-dimensional) ``DataFrame`` structures.
Once you are familiar with indexing and manipulation of data in a ``Series`` and ``DataFrame``, ``Panel`` and ``Panel4D`` are relatively straightforward to use.
In particular, the ``ix``, ``loc``, and ``iloc`` indexers discussed in [Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb) extend readily to these higher-dimensional structures.

We won't cover these panel structures further in this text, as I've found in the majority of cases that multi-indexing is a more useful and conceptually simpler representation for higher-dimensional data.
Additionally, panel data is fundamentally a dense data representation, while multi-indexing is fundamentally a sparse data representation.
As the number of dimensions increases, the dense representation can become very inefficient for the majority of real-world datasets.
For the occasional specialized application, however, these structures can be useful.
If you'd like to read more about the ``Panel`` and ``Panel4D`` structures, see the references listed in [Further Resources](03.13-Further-Resources.ipynb).
![](https://lh3.googleusercontent.com/-ciQqxeG-FpY/XVDFZoRl5hI/AAAAAAAAgfU/3alq5bE4KEoqObtE52gKpj9U_HpAUjDmQCK8BGAs/s512/2019-08-11.png)

In [194]:
df1 = pd.DataFrame({'Fruit':['Apple','Graphs','Chrrey','Kiwi'],
             'Weight':[120.5,105.6,116.8,50.5]})
data = {'Item1':df1,'Item2':df1}
data

{'Item1':     Fruit  Weight
 0   Apple   120.5
 1  Graphs   105.6
 2  Chrrey   116.8
 3    Kiwi    50.5, 'Item2':     Fruit  Weight
 0   Apple   120.5
 1  Graphs   105.6
 2  Chrrey   116.8
 3    Kiwi    50.5}

In [198]:
panel = pd.Panel.from_dict(data,orient='minor')
panel

AttributeError: type object 'Panel' has no attribute 'from_dict'

In [199]:
data = np.random.rand(2,4,5)
p = pd.Panel(data)
print(p)

TypeError: object() takes no parameters

In [200]:
p[0]

TypeError: 'type' object is not subscriptable

In [201]:
p[1]

TypeError: 'type' object is not subscriptable

In [202]:
Data = {'Item1':pd.DataFrame(np.random.rand(4,3)),
       'Item2':pd.DataFrame(np.random.rand(4,2)),
       'Item3':pd.DataFrame(np.random.rand(4,4))}
p = pd.Panel(Data)
print(p)

TypeError: object() takes no parameters

In [203]:
p[0]

TypeError: 'type' object is not subscriptable

In [ ]:
p[1]

In [204]:
print(p.major_xs(1))

AttributeError: type object 'Panel' has no attribute 'major_xs'

In [205]:
p.major_xs(0)

AttributeError: type object 'Panel' has no attribute 'major_xs'

In [ ]:
p.major_xs(2)

In [ ]:
p.major_xs(3)

In [ ]:
p.minor_xs(1)

In [206]:
p.minor_xs(0)

AttributeError: type object 'Panel' has no attribute 'minor_xs'

In [207]:
pd.__version__

'0.25.1'

<!--NAVIGATION-->
< [Handling Missing Data](03.04-Missing-Values.ipynb) | [Contents](Index.ipynb) | [Combining Datasets: Concat and Append](03.06-Concat-And-Append.ipynb) >